In [ ]:
#| default_exp data_prep
%load_ext autoreload
%autoreload 2

In [ ]:
# | exporti
import os
import pandas as pd
import joblib
import yaml
from typing import Union, Final
import numpy as np
from pathlib import Path
from dataclasses import dataclass

In [ ]:
# | export

LINE_FEAT_NAMES: Final[list[str]] = [
    "row",
    "txt_len",
    "end_with_end_sent",
    "end_with_hyphen",
    "start_with_upper",
    "start_with_bullet",
    "line_txt",
    "diff_len_prev",
    "diff_max_len",
]


def create_line_features(lines: list[str]) -> pd.DataFrame:
    """compute features for each lines in a group (doc) and return a dataframe with them

    Arguments:
        lines -- list of lines used to compute features

    Returns:
        the dataframe with the features.
    """
    line_lengths = [len(l) for l in lines]
    line_rows = [i for i, _ in enumerate(lines)]
    are_end_of_sent = [l.strip()[-1] in [".", "?", "!"] if len(l.strip()) > 0 else False for l in lines]
    are_end_hyphen = [l.strip()[-1] in ["-"] if len(l.strip()) > 0 else False for l in lines]
    # erreur car élimine des lignes. ils doit avoir une valeur else
    are_start_upper = [l.strip()[0].isupper() if len(l.strip()) > 0 else False for l in lines]
    are_start_bullet = [l.strip().startswith(("-", "•", "o ")) if len(l.strip()) > 0 else False for l in lines]

    assert (
        len(line_lengths)
        == len(line_rows)
        == len(are_end_of_sent)
        == len(are_end_hyphen)
        == len(are_start_upper)
        == len(are_start_bullet)
    ), "all lines must be processed."

    lines_data = [
        (r, l, e, h, u, b, t)
        for r, t, l, e, h, u, b in zip(
            line_rows,
            lines,
            line_lengths,
            are_end_of_sent,
            are_end_hyphen,
            are_start_upper,
            are_start_bullet,
        )
    ]
    lines_df = pd.DataFrame(
        lines_data,
        columns=[
            "row",
            "txt_len",
            "end_with_end_sent",
            "end_with_hyphen",
            "start_with_upper",
            "start_with_bullet",
            "line_txt",
        ],
    )
    lines_df["diff_len_prev"] = lines_df.txt_len.diff()
    lines_df.diff_len_prev = lines_df.diff_len_prev.fillna(lines_df.txt_len)
    lines_df["diff_max_len"] = lines_df.txt_len.max() - lines_df.txt_len

    return lines_df

In [ ]:
# | export
def prepare_data_from_doc(file_path: Union[str, Path]) -> tuple[pd.DataFrame, list[str]]:
    with open(file_path, "r") as f:
        lines: list[str] = f.read().split("\n")
        line_feats_df = create_line_features(lines)
    return line_feats_df, lines

In [ ]:
prep_df, lines = prepare_data_from_doc(Path("../test_data/doc_a.txt"))
assert all([c in LINE_FEAT_NAMES for c in prep_df.columns.tolist()])


In [ ]:
import nbdev

nbdev.nbdev_export("data_prep.ipynb")
